# PyIceberg Lab - Apache Iceberg with Python

## Lab Objectives
In this lab, we will learn how to:
1. Install and configure PyIceberg
2. Create and manage Iceberg catalogs
3. Create Iceberg tables and manipulate data
4. Perform schema evolution
5. Query and analyze data

## What is Apache Iceberg?
Apache Iceberg is an open, high-performance table format for huge analytic datasets, bringing reliability and simplicity to big data. PyIceberg is the Python library for programmatic access to Iceberg table metadata and data.


## 1. Install PyIceberg

**Note:** If you have already set up the conda environment `datalab` following the instructions in README.md, skip this section and move to the next cell.

If not set up yet, you can install directly:


In [ ]:
# Install PyIceberg with necessary dependencies
# Only run this cell if you haven't set up the conda environment
%pip install --upgrade pip
%pip install "pyiceberg[s3fs,hive,pyarrow]"
%pip install 'pyiceberg[sql-sqlite]
% pip install 'pyiceberg[sql-postgres]'


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: unmatched '
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Import necessary libraries
import os
import tempfile
from pyiceberg.catalog import load_catalog
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pandas as pd


## 2. Create Warehouse Directory and Configure Catalog

We will create a temporary warehouse directory to store Iceberg data:


In [19]:
# Create warehouse directory
warehouse_path = "/tmp/pyiceberg_warehouse"
os.makedirs(warehouse_path, exist_ok=True)
print(f"Created warehouse directory: {warehouse_path}")

# Configure SQL catalog with SQLite
catalog = load_catalog(
    "default",
    **{
        'type': 'sql',
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

print("Successfully configured catalog!")


Created warehouse directory: /tmp/pyiceberg_warehouse
Successfully configured catalog!


## 3. Create Namespace and Prepare Sample Data

Before creating tables, we need to create a namespace and prepare sample data:


In [20]:
# Create namespace 'default'
try:
    catalog.create_namespace("default")
    print("Created namespace 'default'")
except Exception as e:
    print(f"Namespace 'default' already exists or error: {e}")

# Create sample taxi trip data
sample_data = {
    'VendorID': [1, 1, 2, 2, 1],
    'tpep_pickup_datetime': ['2023-01-01 00:00:00', '2023-01-01 00:15:00', '2023-01-01 00:30:00', '2023-01-01 00:45:00', '2023-01-01 01:00:00'],
    'tpep_dropoff_datetime': ['2023-01-01 00:10:00', '2023-01-01 00:25:00', '2023-01-01 00:40:00', '2023-01-01 00:55:00', '2023-01-01 01:10:00'],
    'passenger_count': [1.0, 2.0, 1.0, 3.0, 2.0],
    'trip_distance': [1.5, 2.3, 0.8, 3.2, 1.9],
    'RatecodeID': [1.0, 1.0, 1.0, 1.0, 1.0],
    'store_and_fwd_flag': ['N', 'N', 'N', 'N', 'N'],
    'PULocationID': [1, 2, 3, 4, 5],
    'DOLocationID': [2, 3, 4, 5, 6],
    'payment_type': [1, 1, 2, 1, 2],
    'fare_amount': [5.5, 8.2, 3.2, 12.5, 7.8],
    'extra': [0.5, 0.5, 0.5, 0.5, 0.5],
    'mta_tax': [0.5, 0.5, 0.5, 0.5, 0.5],
    'tip_amount': [1.0, 1.5, 0.5, 2.5, 1.2],
    'tolls_amount': [0.0, 0.0, 0.0, 0.0, 0.0],
    'improvement_surcharge': [0.3, 0.3, 0.3, 0.3, 0.3],
    'total_amount': [7.8, 11.0, 5.0, 16.3, 10.3],
    'congestion_surcharge': [0.0, 0.0, 0.0, 0.0, 0.0],
    'airport_fee': [0.0, 0.0, 0.0, 0.0, 0.0]
}

# Convert to PyArrow Table
df = pa.table(sample_data)
print("Created sample data:")
print(df)


Created namespace 'default'
Created sample data:
pyarrow.Table
VendorID: int64
tpep_pickup_datetime: string
tpep_dropoff_datetime: string
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
----
VendorID: [[1,1,2,2,1]]
tpep_pickup_datetime: [["2023-01-01 00:00:00","2023-01-01 00:15:00","2023-01-01 00:30:00","2023-01-01 00:45:00","2023-01-01 01:00:00"]]
tpep_dropoff_datetime: [["2023-01-01 00:10:00","2023-01-01 00:25:00","2023-01-01 00:40:00","2023-01-01 00:55:00","2023-01-01 01:10:00"]]
passenger_count: [[1,2,1,3,2]]
trip_distance: [[1.5,2.3,0.8,3.2,1.9]]
RatecodeID: [[1,1,1,1,1]]
store_and_fwd_flag: [["N","N","N","N","N"]]
PULocationID: [[1,2,3,4,5]]
DOLocationID: [[2,3,4,5,6]]
payment_type: [[1,1,2,

## 4. Create Iceberg Table

Now we will create an Iceberg table from the sample data schema:


In [21]:
# Create Iceberg table from PyArrow DataFrame schema
table = catalog.create_table(
    "default.taxi_dataset",
    schema=df.schema,
)

print("Successfully created Iceberg table 'default.taxi_dataset'!")
print("\nTable schema:")
print(table.schema())


Successfully created Iceberg table 'default.taxi_dataset'!

Table schema:
table {
  1: VendorID: optional long
  2: tpep_pickup_datetime: optional string
  3: tpep_dropoff_datetime: optional string
  4: passenger_count: optional double
  5: trip_distance: optional double
  6: RatecodeID: optional double
  7: store_and_fwd_flag: optional string
  8: PULocationID: optional long
  9: DOLocationID: optional long
  10: payment_type: optional long
  11: fare_amount: optional double
  12: extra: optional double
  13: mta_tax: optional double
  14: tip_amount: optional double
  15: tolls_amount: optional double
  16: improvement_surcharge: optional double
  17: total_amount: optional double
  18: congestion_surcharge: optional double
  19: airport_fee: optional double
}


## 5. Add Data to Table

Now we will add data to the newly created table:


In [22]:
# Add data to table
table.append(df)

# Check number of records added
result_df = table.scan().to_arrow()
print(f"Added {len(result_df)} records to table")
print("\nData in table:")
print(result_df.to_pandas())


Added 5 records to table

Data in table:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2023-01-01 00:00:00   2023-01-01 00:10:00              1.0   
1         1  2023-01-01 00:15:00   2023-01-01 00:25:00              2.0   
2         2  2023-01-01 00:30:00   2023-01-01 00:40:00              1.0   
3         2  2023-01-01 00:45:00   2023-01-01 00:55:00              3.0   
4         1  2023-01-01 01:00:00   2023-01-01 01:10:00              2.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            1.5         1.0                  N             1             2   
1            2.3         1.0                  N             2             3   
2            0.8         1.0                  N             3             4   
3            3.2         1.0                  N             4             5   
4            1.9         1.0                  N             5             6   

   payment_type  fare_amount  ext

## 6. Query and Analyze Data

We will perform some basic queries on the data:


In [23]:
# Query 1: Get all data
print("=== Query 1: All data ===")
all_data = table.scan().to_arrow()
print(f"Number of records: {len(all_data)}")

# Query 2: Filter by fare_amount > 7
print("\n=== Query 2: Filter by fare_amount > 7 ===")
filtered_data = table.scan(row_filter="fare_amount > 7").to_arrow()
print(f"Number of records with fare_amount > 7: {len(filtered_data)}")
if len(filtered_data) > 0:
    print(filtered_data.to_pandas()[['VendorID', 'fare_amount', 'trip_distance', 'total_amount']])

# Query 3: Filter by trip_distance
print("\n=== Query 3: Filter by trip_distance >= 2 ===")
distance_filtered = table.scan(row_filter="trip_distance >= 2").to_arrow()
print(f"Number of records with trip_distance >= 2: {len(distance_filtered)}")
if len(distance_filtered) > 0:
    print(distance_filtered.to_pandas()[['VendorID', 'trip_distance', 'fare_amount']])


=== Query 1: All data ===
Number of records: 5

=== Query 2: Filter by fare_amount > 7 ===
Number of records with fare_amount > 7: 3
   VendorID  fare_amount  trip_distance  total_amount
0         1          8.2            2.3          11.0
1         2         12.5            3.2          16.3
2         1          7.8            1.9          10.3

=== Query 3: Filter by trip_distance >= 2 ===
Number of records with trip_distance >= 2: 2
   VendorID  trip_distance  fare_amount
0         1            2.3          8.2
1         2            3.2         12.5


## 7. Schema Evolution - Add New Column

One of the powerful features of Iceberg is schema evolution. We will add a new column to the table:

**Important Note:** In Iceberg, you must update the schema before you can add data with new columns. Here's the correct way to do it:


In [24]:
# Add new column 'tip_per_mile' to DataFrame
df_with_new_column = df.append_column("tip_per_mile", pc.divide(df["tip_amount"], df["trip_distance"]))

print("DataFrame with new column 'tip_per_mile':")
print(df_with_new_column.to_pandas())

# UPDATE SCHEMA BEFORE OVERWRITE
# Add new column to table schema
from pyiceberg.schema import Schema
from pyiceberg.types import DoubleType, NestedField

# Get current schema
current_schema = table.schema()

# Create new field for tip_per_mile column
new_field = NestedField(
    field_id=len(current_schema.fields) + 1,  # Next ID
    name="tip_per_mile",
    field_type=DoubleType(),
    required=False  # Can be null
)

# Create new schema with added column
new_schema = Schema(*current_schema.fields, new_field)

# Update table schema
table.update_schema().add_column("tip_per_mile", DoubleType()).commit()

print("\nNew table schema after adding column:")
print(table.schema())


DataFrame with new column 'tip_per_mile':
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2023-01-01 00:00:00   2023-01-01 00:10:00              1.0   
1         1  2023-01-01 00:15:00   2023-01-01 00:25:00              2.0   
2         2  2023-01-01 00:30:00   2023-01-01 00:40:00              1.0   
3         2  2023-01-01 00:45:00   2023-01-01 00:55:00              3.0   
4         1  2023-01-01 01:00:00   2023-01-01 01:10:00              2.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            1.5         1.0                  N             1             2   
1            2.3         1.0                  N             2             3   
2            0.8         1.0                  N             3             4   
3            3.2         1.0                  N             4             5   
4            1.9         1.0                  N             5             6   

   payment_type  fare_amount  ex

In [25]:
# Now we can overwrite with data containing the new column
table.overwrite(df_with_new_column)

print("✅ Successfully overwrote with new schema!")
print("\nData after overwrite:")
result = table.scan().to_arrow()
print(result.to_pandas())


✅ Successfully overwrote with new schema!

Data after overwrite:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2023-01-01 00:00:00   2023-01-01 00:10:00              1.0   
1         1  2023-01-01 00:15:00   2023-01-01 00:25:00              2.0   
2         2  2023-01-01 00:30:00   2023-01-01 00:40:00              1.0   
3         2  2023-01-01 00:45:00   2023-01-01 00:55:00              3.0   
4         1  2023-01-01 01:00:00   2023-01-01 01:10:00              2.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            1.5         1.0                  N             1             2   
1            2.3         1.0                  N             2             3   
2            0.8         1.0                  N             3             4   
3            3.2         1.0                  N             4             5   
4            1.9         1.0                  N             5             6   

   paymen

In [26]:
# Query with new column
print("=== Query with new column 'tip_per_mile' ===")
evolved_data = table.scan().to_arrow()
print("Data with new column:")
print(evolved_data.to_pandas()[['VendorID', 'tip_amount', 'trip_distance', 'tip_per_mile']])

# Filter by tip_per_mile > 0.5
print("\n=== Filter by tip_per_mile > 0.5 ===")
tip_filtered = table.scan(row_filter="tip_per_mile > 0.5").to_arrow()
print(f"Number of records with tip_per_mile > 0.5: {len(tip_filtered)}")
if len(tip_filtered) > 0:
    print(tip_filtered.to_pandas()[['VendorID', 'tip_per_mile', 'fare_amount']])


=== Query with new column 'tip_per_mile' ===
Data with new column:
   VendorID  tip_amount  trip_distance  tip_per_mile
0         1         1.0            1.5      0.666667
1         1         1.5            2.3      0.652174
2         2         0.5            0.8      0.625000
3         2         2.5            3.2      0.781250
4         1         1.2            1.9      0.631579

=== Filter by tip_per_mile > 0.5 ===
Number of records with tip_per_mile > 0.5: 5
   VendorID  tip_per_mile  fare_amount
0         1      0.666667          5.5
1         1      0.652174          8.2
2         2      0.625000          3.2
3         2      0.781250         12.5
4         1      0.631579          7.8


### 💡 Explanation of Schema Evolution

**Why do we need to update schema first?**

1. **Iceberg requires explicit schema evolution**: Cannot add data with new columns without updating schema first
2. **Type safety**: Iceberg ensures type consistency across snapshots
3. **Metadata management**: Schema changes are tracked in metadata files

**Ways to perform schema evolution:**

```python
# Method 1: Using update_schema() (Recommended)
table.update_schema().add_column("new_column", DoubleType()).commit()

# Method 2: Using union_by_name (for PyArrow)
pa_table = pa.table.union_by_name([old_table, new_table])

# Method 3: Create completely new schema
new_schema = Schema(*old_fields, new_field)
```


## 8. Snapshot Management and History

Iceberg supports time travel and snapshot management. Let's look at the table history:


In [27]:
# View snapshot history
print("=== Snapshot History ===")
history = table.inspect.history()
print(history.to_pandas())

# View file information in table
print("\n=== File Information ===")
files_info = table.inspect.files()
print(f"Number of files: {len(files_info)}")
if len(files_info) > 0:
    print("File information:")
    print(files_info.to_pandas()[['file_path', 'record_count', 'file_size_in_bytes']])


=== Snapshot History ===
          made_current_at          snapshot_id     parent_id  \
0 2025-09-17 04:10:01.752  3823500475119804241           NaN   
1 2025-09-17 04:10:04.013  4201636063013981244  3.823500e+18   
2 2025-09-17 04:10:04.026  5571586115741167391  4.201636e+18   

   is_current_ancestor  
0                 True  
1                 True  
2                 True  

=== File Information ===
Number of files: 1
File information:
                                           file_path  record_count  \
0  file:///tmp/pyiceberg_warehouse/default/taxi_d...             5   

   file_size_in_bytes  
0                8438  


## 9. Add New Data and Append Operations

We will add more new data to the table to demonstrate append operations:


In [28]:
# Create new data to append
new_sample_data = {
    'VendorID': [2, 1, 2],
    'tpep_pickup_datetime': ['2023-01-01 01:15:00', '2023-01-01 01:30:00', '2023-01-01 01:45:00'],
    'tpep_dropoff_datetime': ['2023-01-01 01:25:00', '2023-01-01 01:40:00', '2023-01-01 01:55:00'],
    'passenger_count': [1.0, 2.0, 1.0],
    'trip_distance': [2.1, 1.7, 2.8],
    'RatecodeID': [1.0, 1.0, 1.0],
    'store_and_fwd_flag': ['N', 'N', 'N'],
    'PULocationID': [6, 7, 8],
    'DOLocationID': [7, 8, 9],
    'payment_type': [1, 2, 1],
    'fare_amount': [9.2, 6.8, 11.5],
    'extra': [0.5, 0.5, 0.5],
    'mta_tax': [0.5, 0.5, 0.5],
    'tip_amount': [1.8, 1.2, 2.3],
    'tolls_amount': [0.0, 0.0, 0.0],
    'improvement_surcharge': [0.3, 0.3, 0.3],
    'total_amount': [12.3, 9.3, 15.1],
    'congestion_surcharge': [0.0, 0.0, 0.0],
    'airport_fee': [0.0, 0.0, 0.0],
    'tip_per_mile': [0.86, 0.71, 0.82]  # Calculate tip_per_mile for new data
}

# Convert to PyArrow Table
new_df = pa.table(new_sample_data)

# Append new data
table.append(new_df)

# Check total records after append
final_data = table.scan().to_arrow()
print(f"Total records after append: {len(final_data)}")
print("\nFinal data:")
print(final_data.to_pandas())


Total records after append: 8

Final data:
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 01:15:00   2023-01-01 01:25:00              1.0   
1         1  2023-01-01 01:30:00   2023-01-01 01:40:00              2.0   
2         2  2023-01-01 01:45:00   2023-01-01 01:55:00              1.0   
3         1  2023-01-01 00:00:00   2023-01-01 00:10:00              1.0   
4         1  2023-01-01 00:15:00   2023-01-01 00:25:00              2.0   
5         2  2023-01-01 00:30:00   2023-01-01 00:40:00              1.0   
6         2  2023-01-01 00:45:00   2023-01-01 00:55:00              3.0   
7         1  2023-01-01 01:00:00   2023-01-01 01:10:00              2.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            2.1         1.0                  N             6             7   
1            1.7         1.0                  N             7             8   
2            2.8         1.0                

## 10. Statistical Analysis and Summary

Finally, we will perform some basic statistical analysis:


In [29]:
# Basic statistical analysis
final_df = final_data.to_pandas()

print("=== OVERVIEW STATISTICS ===")
print(f"Total trips: {len(final_df)}")
print(f"Total revenue: ${final_df['total_amount'].sum():.2f}")
print(f"Average revenue: ${final_df['total_amount'].mean():.2f}")
print(f"Average tip: ${final_df['tip_amount'].mean():.2f}")
print(f"Average distance: {final_df['trip_distance'].mean():.2f} miles")

print("\n=== VENDOR ANALYSIS ===")
vendor_stats = final_df.groupby('VendorID').agg({
    'total_amount': ['count', 'sum', 'mean'],
    'tip_amount': 'mean',
    'trip_distance': 'mean'
}).round(2)
print(vendor_stats)

print("\n=== TOP 3 TRIPS WITH HIGHEST TIPS ===")
top_tips = final_df.nlargest(3, 'tip_amount')[['VendorID', 'tip_amount', 'trip_distance', 'total_amount']]
print(top_tips)

print("\n=== TIP PER MILE ANALYSIS ===")
print(f"Average tip per mile: {final_df['tip_per_mile'].mean():.2f}")
print(f"Highest tip per mile: {final_df['tip_per_mile'].max():.2f}")
print(f"Lowest tip per mile: {final_df['tip_per_mile'].min():.2f}")


=== OVERVIEW STATISTICS ===
Total trips: 8
Total revenue: $87.10
Average revenue: $10.89
Average tip: $1.50
Average distance: 2.04 miles

=== VENDOR ANALYSIS ===
         total_amount              tip_amount trip_distance
                count   sum   mean       mean          mean
VendorID                                                   
1                   4  38.4   9.60       1.23          1.85
2                   4  48.7  12.18       1.78          2.22

=== TOP 3 TRIPS WITH HIGHEST TIPS ===
   VendorID  tip_amount  trip_distance  total_amount
6         2         2.5            3.2          16.3
2         2         2.3            2.8          15.1
0         2         1.8            2.1          12.3

=== TIP PER MILE ANALYSIS ===
Average tip per mile: 0.72
Highest tip per mile: 0.86
Lowest tip per mile: 0.62


## 11. Check Warehouse Structure

Finally, let's examine the warehouse structure to understand how Iceberg stores data:


In [30]:
# Check warehouse structure
import os

print("=== WAREHOUSE STRUCTURE ===")
print(f"Warehouse path: {warehouse_path}")

def explore_directory(path, prefix=""):
    """Recursively explore directory structure"""
    try:
        items = os.listdir(path)
        for item in sorted(items):
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                print(f"{prefix}📁 {item}/")
                explore_directory(item_path, prefix + "  ")
            else:
                size = os.path.getsize(item_path)
                print(f"{prefix}📄 {item} ({size} bytes)")
    except PermissionError:
        print(f"{prefix}❌ Permission denied")

explore_directory(warehouse_path)


=== WAREHOUSE STRUCTURE ===
Warehouse path: /tmp/pyiceberg_warehouse
📁 default/
  📁 taxi_dataset/
    📁 data/
      📄 00000-0-b158b449-56df-46b2-9dc8-f8c9fcee9276.parquet (8016 bytes)
      📄 00000-0-d4877cfd-8619-440e-9594-f4d8a4777f26.parquet (8438 bytes)
      📄 00000-0-d5914a35-e2a6-4a5e-85a6-b069b1dd879d.parquet (8343 bytes)
    📁 metadata/
      📄 00000-b7aea812-9611-4e0b-b606-c1ffca6a1b9b.metadata.json (1836 bytes)
      📄 00001-b795de7b-9b6f-4505-991e-7965afd53ab4.metadata.json (2668 bytes)
      📄 00002-fe12bb19-a23e-4499-9741-0e18fdd7f677.metadata.json (4210 bytes)
      📄 00003-1d8e8f2c-2da2-41b2-8e22-f88b84ceb708.metadata.json (5597 bytes)
      📄 00004-f631d466-6264-4ade-9b47-9029a1decbbb.metadata.json (6374 bytes)
      📄 091cfe63-6336-4a48-817a-3aa3a4e5503b-m0.avro (5737 bytes)
      📄 b158b449-56df-46b2-9dc8-f8c9fcee9276-m0.avro (5672 bytes)
      📄 d4877cfd-8619-440e-9594-f4d8a4777f26-m0.avro (5754 bytes)
      📄 d5914a35-e2a6-4a5e-85a6-b069b1dd879d-m0.avro (5759 bytes

## Conclusion

In this lab, we have learned:

### ✅ What we accomplished:
1. **Installed PyIceberg** with necessary dependencies
2. **Configured SQL Catalog** with SQLite for development
3. **Created Iceberg table** from PyArrow schema
4. **Added and managed data** with append operations
5. **Performed queries** with filtering
6. **Schema evolution** - added new column to table
7. **Snapshot management** and history tracking
8. **Statistical analysis** of taxi trip data

### 🎯 Key features of Apache Iceberg:
- **Schema Evolution**: Add/remove/modify columns without rewriting entire data
- **Time Travel**: Access data at different points in time
- **ACID Transactions**: Ensure data consistency
- **Partitioning**: Optimize query performance
- **Metadata Management**: Efficient metadata management

### 🚀 Next steps:
- Experiment with larger datasets
- Configure with cloud storage (S3, GCS, Azure)
- Integrate with other query engines (Spark, Trino, etc.)
- Implement partitioning strategies

**Congratulations! You have completed the basic PyIceberg lab! 🎉**
